# S-matrix theory

In [1]:
import numpy as np
import matplotlib as plt
import sympy as sp

from logzero import logger
from IPython.display import display, Math

%matplotlib widget

# Deriving the Redheffer star product

In [19]:
x1, x2, x3, x4, x5, x6 = sp.symbols( ' '.join( [ 'x_%d'%i for i in range( 1, 7 ) ] ) )
A11, A12, A21, A22 = sp.symbols( ' '.join( [ 'A_{%d%d}'%( i, j ) for i in [1,2] for j in [1,2] ] ) )
B11, B12, B21, B22 = sp.symbols( ' '.join( [ 'B_{%d%d}'%( i, j ) for i in [1,2] for j in [1,2] ] ) )

eq1 = A11*x1 + A12*x5 - x6
eq2 = B21*x6 + B22*x2 - x5
intermediate_solution = sp.linsolve( [ eq1, eq2  ], ( x5, x6 ) )
x5, x6 = list( intermediate_solution )[0]
logger.info( 'Intermediate solution: ' )
for exp, lbl in zip( [ x5, x6 ], [ 'x_5', 'x_6' ] ):
    display( Math( f'\\Large{sp.latex( exp )}' ) )

[I 250901 17:44:48 1224141417:9] Intermediate solution: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
eq3 = B11*x6 + B12*x2 - x3
eq4 = A21*x1 + A22*x5 - x4
final_solution = sp.linsolve( [ eq3, eq4 ], ( x3, x4 ) )
x3, x4 = list( final_solution )[0]
logger.info( 'Final solution: ' )
for exp, lbl in zip( [ x3, x4 ], [ 'x_3', 'x_4' ] ):
    display( Math( f'\\Large{sp.latex( sp.simplify( exp ) )}' ) )

eq5 = list( final_solution )[0][0] - sp.symbols( 'x3' )
eq6 = list( final_solution )[0][1] - sp.symbols( 'x4' )
S, coeff = sp.linear_eq_to_matrix( [ eq5, eq6 ], ( x1, x2 ) )

[I 250901 17:34:32 328423006:5] Final solution: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
logger.info( 'Final S-matrix: ' )
Sl = sp.latex(sp.simplify(S))
display( Math( f'\\Large{sp.latex(sp.simplify(S))}' ) )

[I 250831 23:58:31 283930408:1] Final S-matrix: 


<IPython.core.display.Math object>

# S-matrix elements for 1-D scattering

In [4]:
t01, t12, t10, t21, r01, r12, phi, phi1, phi2 = sp.symbols( 
    't_{01} t_{12} t_{10} t_{21} r_{01} r_{12} phi phi1 phi2'
) # reflection coefficients are sign-reversed. 

In [5]:
def S11(t01, t12, r01, r12, phi):
    return (t01 * t12) * sp.exp(sp.I * phi) / (1 + r01 * r12 * sp.exp(sp.I * 2 * phi) )
    # return (t01 * t12) * sp.exp(sp.I * phi) / (1 + r01 * r12 ) # * sp.exp(sp.I * 2 * phi) )

S11(t01, t12, r01, r12, phi)

t_{01}*t_{12}*exp(I*phi)/(r_{01}*r_{12}*exp(2*I*phi) + 1)

In [6]:
def S12(r12, t21, r01, t12, phi):
    return -r12 + ( -r01 * t21 * t12 * sp.exp( -sp.I*2*phi) ) / ( 1 + r01*r12*sp.exp( -sp.I*2*phi ) )
    # return -r12 + ( -r01*t21*t12 ) / ( 1 + r01*r12 ) #*sp.exp( -sp.I*2*phi ) )

S12(r12, t21, r01, t12, phi)

-r_{01}*t_{12}*t_{21}*exp(-2*I*phi)/(r_{01}*r_{12}*exp(-2*I*phi) + 1) - r_{12}

In [7]:
def S21( r01, t01, r12, t10, phi ):
    return r01 + (t01*r12*t10 * sp.exp( sp.I*2*phi ) ) / (1 +r01*r12*sp.exp( sp.I*2*phi ) )
    # return r01 + (t01*r12*t10 ) / (1 +r01*r12 ) #*sp.exp( sp.I*2*phi ) )

S21( r01, t01, r12, t10, phi )

r_{01} + r_{12}*t_{01}*t_{10}*exp(2*I*phi)/(r_{01}*r_{12}*exp(2*I*phi) + 1)

In [8]:
def S22( t21, t10, r01, r12, phi ):
    return (t21*t10 )*sp.exp( -sp.I*phi ) / ( 1 + r01*r12*sp.exp( -sp.I*2*phi ) )

S22( t21, t10, r01, r12, phi )

t_{10}*t_{21}*exp(-I*phi)/(r_{01}*r_{12}*exp(-2*I*phi) + 1)

In [9]:
def S( t01, t10, t12, t21, r01, r12, phi ):
    return sp.Matrix( 
        [ 
            [ S11( t01, t12, r01, r12, phi ), S12( r12, t21, r01, t12, phi ) ], 
            [ S21( r01, t01, r12, t10, phi ), S22( t21, t10, r01, r12, phi ) ]
        ]
    )

S( t01, t10, t12, t21, r01, r12, phi )

Matrix([
[                  t_{01}*t_{12}*exp(I*phi)/(r_{01}*r_{12}*exp(2*I*phi) + 1), -r_{01}*t_{12}*t_{21}*exp(-2*I*phi)/(r_{01}*r_{12}*exp(-2*I*phi) + 1) - r_{12}],
[r_{01} + r_{12}*t_{01}*t_{10}*exp(2*I*phi)/(r_{01}*r_{12}*exp(2*I*phi) + 1),                    t_{10}*t_{21}*exp(-I*phi)/(r_{01}*r_{12}*exp(-2*I*phi) + 1)]])

# Sanity checks

## Free-space transmission

In [10]:
S( 1, 1, 1, 1, 0, 0, phi )

Matrix([
[exp(I*phi),           0],
[         0, exp(-I*phi)]])

In [11]:
sp.simplify( S( 1, 1, 1, 1, 0, 0, phi2 ) @ S( 1, 1, 1, 1, 0, 0, phi1 ) )

Matrix([
[exp(I*(phi1 + phi2)),                     0],
[                   0, exp(I*(-phi1 - phi2))]])

# Concatenating identical blocks

In [12]:
S1 = S( t01, t10, 1, 1, r01, 0, phi )
S2 = S( 1, 1, t10, t01, 0, -r01, phi )

In [13]:
S1

Matrix([
[t_{01}*exp(I*phi), -r_{01}*exp(-2*I*phi)],
[           r_{01},    t_{10}*exp(-I*phi)]])

In [14]:
S2

Matrix([
[   t_{10}*exp(I*phi),             r_{01}],
[-r_{01}*exp(2*I*phi), t_{01}*exp(-I*phi)]])

In [15]:
S2 @ S1

Matrix([
[                 r_{01}**2 + t_{01}*t_{10}*exp(2*I*phi),                                       0],
[-r_{01}*t_{01}*exp(3*I*phi) + r_{01}*t_{01}*exp(-I*phi), r_{01}**2 + t_{01}*t_{10}*exp(-2*I*phi)]])

In [16]:
S( t01, t10, t10, t01, r01, -r01, 2*phi )

Matrix([
[                 t_{01}*t_{10}*exp(2*I*phi)/(-r_{01}**2*exp(4*I*phi) + 1), -r_{01}*t_{01}*t_{10}*exp(-4*I*phi)/(-r_{01}**2*exp(-4*I*phi) + 1) + r_{01}],
[-r_{01}*t_{01}*t_{10}*exp(4*I*phi)/(-r_{01}**2*exp(4*I*phi) + 1) + r_{01},                  t_{01}*t_{10}*exp(-2*I*phi)/(-r_{01}**2*exp(-4*I*phi) + 1)]])